0: Import Libraries

In [1]:
# Check Python version
import sys
print(f"Python version: {sys.version}")

# Update package index and install system dependencies in one step
!apt-get update -qq && apt-get install -y libreoffice poppler-utils tesseract-ocr tesseract-ocr-eng -qq
# - libreoffice: For DOCX to PDF conversion
# - poppler-utils: For pdf2image
# - tesseract-ocr & tesseract-ocr-eng: For OCR functionality

# Install core Python dependencies in a single command
!pip install --quiet --no-cache-dir \
    torch torchvision \
    pdf2image opencv-python-headless \
    pytesseract PyPDF2 \
    python-docx \
    pillow \
    cython pycocotools fvcore \
    sentence-transformers \
    fastapi uvicorn pyngrok \
    psycopg2-binary

!pip install protobuf==3.20.3 google-api-core==1.34.1

!pip install faiss-cpu --upgrade
# Install detectron2 from a specific commit for stability and compatibility
!pip install --quiet -U 'git+https://github.com/facebookresearch/detectron2.git@ff53992b1985b63bd3262b5a36167098e3dada02'

# Install layoutparser with OCR support
!pip install --quiet "layoutparser[ocr]"

# Verify installations (optional, for debugging)
import torch
import layoutparser
import detectron2
import faiss
print(f"Torch version: {torch.__version__}")
print(f"Layoutparser version: {layoutparser.__version__}")
print(f"Detectron2 version: {detectron2.__version__}")
print(f"FAISS version: {faiss.__version__}")

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
  Preparing metadata (setup.py) ... done
Torch version: 2.6.0+cu124
Layoutparser version: 0.3.4
Detectron2 version: 0.6
FAISS version: 1.10.0


1: Imports and Initial Setup

In [35]:
# Import necessary libraries for deep learning, computer vision, OCR, and database operations
import torch
import torchvision
import layoutparser as lp
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.model_zoo import model_zoo
import os
import subprocess
import json
import traceback
from datetime import datetime
from pathlib import Path
from pdf2image import convert_from_path
import cv2
import pytesseract
import numpy as np
from PIL import Image
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
import re
from sentence_transformers import SentenceTransformer
import faiss
import psycopg2
from psycopg2.extras import execute_values
from google.colab import drive

# Configuration flag for database usage
USE_POSTGRES = True

# Neon PostgreSQL connection details
DB_CONFIG = {
    "dbname": "neondb",
    "user": "neondb_owner",
    "password": "npg_DU3Vxoi6cCIu",
    "host": "ep-purple-sound-a4z952yb-pooler.us-east-1.aws.neon.tech",
    "port": "5432",
    "sslmode": "require"
}

2: Device and Directory Configuration

In [36]:
# Select device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Mount Google Drive for file storage
drive.mount('/content/drive')

from datetime import datetime
import shutil

# Define base directory and subdirectories for file organization
BASE_DIR = "/content/drive/My Drive/lifesciences"
SOURCE_DIR = Path(BASE_DIR) / "training_documents"
PDF_STAGING_DIR = Path(BASE_DIR) / "staging/pdf"
IMG_STAGING_DIR = Path(BASE_DIR) / "post_label/images"
ANNOTATED_DIR = Path(BASE_DIR) / "staging/annotated"
MODELS_DIR = Path(BASE_DIR) / "models"
RETRAINED_STAGING_DIR = Path(BASE_DIR) / "retrained_staging"
RETRAINED_ANNOTATED_DIR = RETRAINED_STAGING_DIR / "annotated"
FAISS_INDEX_PATH = Path(BASE_DIR) / "faiss_index_main.index"  # Single main index
FAISS_BACKUP_DIR = Path(BASE_DIR) / "faiss_backups"  # Directory for backups
COCO_JSON_PATH = Path(BASE_DIR) / "post_label/result.json"
RETRAINED_MODELS_DIR = Path(BASE_DIR) / "retrained_staging/models"
# Create directories if they don’t exist
for directory in [SOURCE_DIR, PDF_STAGING_DIR, IMG_STAGING_DIR, ANNOTATED_DIR, MODELS_DIR, RETRAINED_STAGING_DIR, RETRAINED_ANNOTATED_DIR, FAISS_BACKUP_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Increase maximum image pixel limit to handle large images
Image.MAX_IMAGE_PIXELS = 100000000

Using device: cpu
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


3: Global Variables and Constants

In [37]:
# Initialize global variables
#id_seq = 1 removed this as we will use database sequence
sentence_model = None
faiss_index = None
model = None
ocr_agent = None
db_conn = None
db_cursor = None
run_number = 1
batch_run_id = None
run_date_time = datetime.now().isoformat()
logged_in_user = "admin"
FAISS_INDEX_PATH = Path(BASE_DIR) / "faiss_index_retrained.index"
# Define colors for different block types for visualization
COLORS = {
    "title": (255, 0, 0), "text": (0, 255, 0), "list": (0, 0, 255),
    "table": (255, 255, 0), "figure": (255, 0, 255)
}

# Map numerical labels to block types
LABEL_MAP = {0: "figure", 1: "list", 2: "table", 3: "text", 4: "title"}

4: COCO JSON Path Mapping

In [38]:
# Function to update COCO JSON file paths based on actual image directory
def map_and_update_coco_paths(json_path, image_dir):
    try:
        with open(json_path, "r") as f:
            coco_data = json.load(f)
    except Exception as e:
        print(f"Error loading JSON {json_path}: {e}")
        raise

    # Create a mapping of actual image filenames
    actual_images = {f: f for f in os.listdir(image_dir) if f.endswith(".png")}
    print(f"Actual images in {image_dir}: {list(actual_images.keys())}")

    updated_images = []
    for image in coco_data["images"]:
        original_name = image["file_name"]
        cleaned_name = os.path.basename(original_name.replace("./images/", ""))
        if cleaned_name in actual_images:
            image["file_name"] = os.path.join(image_dir, cleaned_name)
            updated_images.append(image)
            print(f"Mapped {original_name} to {image['file_name']}")
        else:
            print(f"No match found for {cleaned_name} in {image_dir}")

    coco_data["images"] = updated_images
    updated_json_path = RETRAINED_STAGING_DIR / "results_mapped.json"
    with open(updated_json_path, "w") as f:
        json.dump(coco_data, f, indent=2)
    print(f"Updated COCO JSON saved to {updated_json_path}")
    return updated_json_path

5: Dataset Registration

In [39]:
# Function to register a COCO dataset with Detectron2
def register_coco_dataset(json_path, image_dir):
    dataset_name = "custom_layout_dataset"
    try:
        # Remove existing dataset if it exists
        if dataset_name in DatasetCatalog:
            DatasetCatalog.remove(dataset_name)
            if dataset_name in MetadataCatalog:
                MetadataCatalog.remove(dataset_name)
            print(f"Unregistered existing dataset: {dataset_name}")

        # Register the new dataset
        register_coco_instances(dataset_name, {}, str(json_path), image_dir)
        MetadataCatalog.get(dataset_name).thing_classes = list(LABEL_MAP.values())
        print(f"Registered dataset: {dataset_name}")
    except Exception as e:
        print(f"Error registering dataset: {e}")
        raise
    return dataset_name

6: Model Training

In [40]:
# Function to train the LayoutParser model using Detectron2
def train_layoutparser_model(json_path, image_dir):
    global model
    dataset_name = register_coco_dataset(json_path, image_dir)
    cfg = get_cfg()

    # Load pre-trained model if available, otherwise use model zoo weights
    pretrained_model_path = os.path.join(MODELS_DIR, "model_final.pth")
    if os.path.exists(pretrained_model_path):
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
        cfg.MODEL.WEIGHTS = pretrained_model_path
        print(f"Loaded pre-trained weights from {pretrained_model_path} (Faster R-CNN)")
    else:
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
        print("Using default Detectron2 Faster R-CNN model zoo weights")

    # Configure training parameters
    cfg.MODEL.DEVICE = str(device)
    cfg.DATASETS.TRAIN = (dataset_name,)
    cfg.DATASETS.TEST = ()
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = 300
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(LABEL_MAP)
    cfg.OUTPUT_DIR = str(RETRAINED_MODELS_DIR)

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    # Save the trained model and configuration
    model_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    config_path = os.path.join(cfg.OUTPUT_DIR, "config.yaml")
    with open(config_path, "w") as f:
        f.write(cfg.dump())

    model = lp.Detectron2LayoutModel(
        config_path=config_path,
        model_path=model_path,
        label_map=LABEL_MAP,
        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.2],
        device=str(device)
    )
    print(f"Model retrained and saved to {model_path}")

7: Lazy Loading Models

In [41]:
# Function to lazily load OCR and layout models
def lazy_load_models():
    global model, ocr_agent
    if ocr_agent is None:
        try:
            ocr_agent = lp.TesseractAgent(languages='eng', config='--psm 6 --oem 1')
            print("OCR agent loaded")
        except Exception as e:
            print(f"Error loading OCR agent: {e}")
            raise
    if model is None:
        model_path = os.path.join(RETRAINED_MODELS_DIR, "model_final.pth")
        config_path = os.path.join(RETRAINED_MODELS_DIR, "config.yaml")
        if os.path.exists(model_path) and os.path.exists(config_path):
            try:
                model = lp.Detectron2LayoutModel(
                    config_path=config_path,
                    model_path=model_path,
                    label_map=LABEL_MAP,
                    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.2],
                    device=str(device)
                )
                print(f"Loaded retrained model from {model_path}")
            except Exception as e:
                print(f"Error loading model: {e}")
                raise
        else:
            print(f"No retrained model or config found at {model_path} or {config_path}. Training required.")

8: Database Connection Setup

In [42]:
# Function to set up PostgreSQL database connection
def setup_db_connection():
    global db_conn, db_cursor, run_number, batch_run_id
    if db_conn is None:
        try:
            db_conn = psycopg2.connect(**DB_CONFIG)
            db_cursor = db_conn.cursor()
            print("Connected to Neon PostgreSQL database")
            insert_batch_query = """
                INSERT INTO batch_runs (run_number, started_at, processed_files, batch_status)
                VALUES ((SELECT COALESCE(MAX(run_number), 0) + 1 FROM batch_runs), %s, 0, 'RUNNING')
                RETURNING id, run_number
            """
            db_cursor.execute(insert_batch_query, (datetime.now().isoformat(),))
            batch_run_id, run_number = db_cursor.fetchone()
            db_conn.commit()
            print(f"Started new batch run: run_number={run_number}, batch_run_id={batch_run_id}, status=RUNNING")
        except Exception as e:
            print(f"Failed to connect to Neon PostgreSQL: {e}")
            raise

# Function to update the run status in the database
def update_run_status(status):
    global db_conn, db_cursor, batch_run_id
    if db_conn is not None:
        try:
            update_query = """
                UPDATE batch_runs
                SET completed_at = %s,
                    processed_files = (SELECT COUNT(DISTINCT source_file_id) FROM source_file_runs WHERE batch_run_id = %s),
                    batch_status = %s
                WHERE id = %s
            """
            db_cursor.execute(update_query, (datetime.now().isoformat(), batch_run_id, status, batch_run_id))
            db_conn.commit()
            print(f"Updated batch run {batch_run_id} status to {status} with processed files count")
        except Exception as e:
            print(f"Failed to update run status: {e}")
            db_conn.rollback()

This function populates documents, source_files, and source_file_runs for each file processed.

In [43]:
def process_source_file_metadata(source_file_path):
    global db_conn, db_cursor, batch_run_id  # Use batch_run_id instead of run_number
    setup_db_connection()

    try:
        file_path = Path(source_file_path)
        document_name = file_path.stem  # e.g., "URS_System_A"
        file_name = file_path.stem  # e.g., "URS_System_A_v1"
        file_type = file_path.suffix[1:] if file_path.suffix in ('.docx', '.pdf') else 'other'

        # Insert or get document
        db_cursor.execute("""
            INSERT INTO documents (name)
            VALUES (%s)
            ON CONFLICT (name) DO UPDATE SET name = EXCLUDED.name
            RETURNING id
        """, (document_name,))
        document_id = db_cursor.fetchone()[0]

        # Insert or get source file
        db_cursor.execute("""
            INSERT INTO source_files (document_id, file_name, file_type)
            VALUES (%s, %s, %s)
            ON CONFLICT (document_id, file_name) DO UPDATE SET file_type = EXCLUDED.file_type
            RETURNING id
        """, (document_id, file_name, file_type))
        source_file_id = db_cursor.fetchone()[0]

        # Extract metadata from the source file
        author = "Unknown"
        created_dt = datetime.now().date()
        last_modified_dt = datetime.now().date()
        number_pages = 1

        if file_type == 'pdf':
            from pdf2image import convert_from_path
            number_pages = len(convert_from_path(source_file_path))
            try:
                from PyPDF2 import PdfReader
                pdf = PdfReader(source_file_path)
                info = pdf.metadata
                if info:
                    author = info.get('/Author', 'Unknown') or 'Unknown'
                    if info.get('/CreationDate'):
                        created_dt = datetime.strptime(info['/CreationDate'][2:10], '%Y%m%d').date()
                    if info.get('/ModDate'):
                        last_modified_dt = datetime.strptime(info['/ModDate'][2:10], '%Y%m%d').date()
            except ImportError:
                print("PyPDF2 not installed; skipping detailed PDF metadata extraction")
        elif file_type == 'docx':
            doc = Document(source_file_path)
            number_pages = len(doc.sections)  # Rough estimate
            props = doc.core_properties
            author = props.author or 'Unknown'
            created_dt = props.created.date() if props.created else datetime.now().date()
            last_modified_dt = props.modified.date() if props.modified else datetime.now().date()

        # Use batch_run_id (references batch_runs.id) instead of run_number
        db_cursor.execute("""
            INSERT INTO source_file_runs (source_file_id, batch_run_id, author, created_dt, last_modified_dt, number_pages)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (source_file_id, batch_run_id) DO UPDATE
                SET author = EXCLUDED.author,
                    created_dt = EXCLUDED.created_dt,
                    last_modified_dt = EXCLUDED.last_modified_dt,
                    number_pages = EXCLUDED.number_pages
            RETURNING id
        """, (source_file_id, batch_run_id, author, created_dt, last_modified_dt, number_pages))
        source_file_run_id = db_cursor.fetchone()[0]

        db_conn.commit()
        print(f"Processed source file metadata for {source_file_path}: source_file_run_id = {source_file_run_id}")
        return source_file_run_id, document_name
    except Exception as e:
        print(f"Error processing source file metadata: {e}")
        db_conn.rollback()
        raise

In [44]:
def append_skipped_blocks_to_db(skipped_blocks):
    if not skipped_blocks:
        return

    setup_db_connection()
    try:
        MAX_CONTENT_LENGTH = 1000
        values = [
            (
                batch_run_id,  # Use batch_run_id instead of run_number
                sb["parent_block_type"],
                sb["skipped_block_type"],
                sb["section_name"][:255] if sb["section_name"] else None,
                sb["parent_block_content"][:MAX_CONTENT_LENGTH-3] + "..." if len(sb["parent_block_content"]) > MAX_CONTENT_LENGTH else sb["parent_block_content"],
                sb["skipped_block_content"][:MAX_CONTENT_LENGTH-3] + "..." if len(sb["skipped_block_content"]) > MAX_CONTENT_LENGTH else sb["skipped_block_content"],
                sb["parent_block_coordinates"],
                sb["skipped_block_coordinates"]
            )
            for sb in skipped_blocks
        ]
        query = """
            INSERT INTO skipped_block_items (
                batch_run_id, parent_block_type, skipped_block_type, section_name,
                parent_block_content, skipped_block_content,
                parent_block_coordinates, skipped_block_coordinates
            ) VALUES %s
        """
        execute_values(db_cursor, query, values)
        db_conn.commit()
        print(f"Inserted {len(skipped_blocks)} skipped blocks into skipped_block_items")
    except Exception as e:
        print(f"Skipped blocks database error: {e}")
        db_conn.rollback()

9: Layout Processing

In [45]:
# Function to check if two rectangles overlap
def is_overlapping(rect1, rect2):
    return not (rect1[2] <= rect2[0] or rect2[2] <= rect1[0] or
                rect1[3] <= rect2[1] or rect2[3] <= rect1[1])

def process_layout(image_path, original_file_path):
    global model
    lazy_load_models()
    if model is None:
        raise ValueError("Model not loaded. Train or load a model first.")

    source_file_run_id, document_name = process_source_file_metadata(original_file_path)
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")

    layout = model.detect(image)
    sorted_layout = sorted(layout, key=lambda x: x.coordinates[1])
    results = []
    skipped_blocks = []
    list_table_regions = [(b.type.lower(), list(map(int, b.coordinates))) for b in sorted_layout if b.type.lower() in ["list", "table"]]
    current_section = "No Title"
    block_number = 1

    list_table_contents = {}
    for block_type, coords in list_table_regions:
        cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
        content = ocr_agent.detect(cropped) or ""
        if block_type == "list":
            content = re.sub(r'[^a-zA-Z0-9\s]', '', content)
        elif block_type == "table":
            content = '\n'.join('-'.join(col.strip() for col in row.split() if col.strip()) for row in content.split('\n') if row.strip()) or "Table Content"
        list_table_contents[tuple(coords)] = content.strip()
        del cropped

    for block in sorted_layout:
        coords = list(map(int, block.coordinates))
        block_type = block.type.lower()
        overlapping_region = next((r for r in list_table_regions if is_overlapping(coords, r[1])), None)
        if block_type in ["title", "text"] and overlapping_region:
            parent_type, parent_coords = overlapping_region
            cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
            skipped_content = ocr_agent.detect(cropped) or ""
            if block_type == "title":
                skipped_content = skipped_content or "Untitled"
            print(f"Skipping {block_type} inside {parent_type} at coordinates: {coords}")
            skipped_blocks.append({
                "parent_block_type": parent_type,
                "skipped_block_type": block_type,
                "section_name": current_section,
                "parent_block_content": list_table_contents[tuple(parent_coords)],
                "skipped_block_content": skipped_content.strip(),
                "parent_block_coordinates": parent_coords,
                "skipped_block_coordinates": coords
            })
            del cropped
            continue

        cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
        if block_type in ["list", "table"]:
            content = list_table_contents[tuple(coords)]
        elif block_type == "title":
            content = ocr_agent.detect(cropped) or "Untitled"
            current_section = content.lower()
        else:
            content = ocr_agent.detect(cropped) or ""

        db_cursor.execute("""
            INSERT INTO sections (section_name)
            VALUES (%s)
            ON CONFLICT (section_name) DO UPDATE SET section_name = EXCLUDED.section_name
            RETURNING id
        """, (current_section,))
        section_id = db_cursor.fetchone()[0]

        db_cursor.execute("""
            INSERT INTO source_file_run_sections (source_file_run_id, section_id)
            VALUES (%s, %s)
            ON CONFLICT DO NOTHING
        """, (source_file_run_id, section_id))

        # Remove 'id' from result dict
        result = {
            "source_file_run_id": source_file_run_id,
            "section_id": section_id,
            "block_number": block_number,
            "content_type": block_type,
            "content": content.strip(),
            "coord_x1": coords[0],
            "coord_y1": coords[1],
            "coord_x2": coords[2],
            "coord_y2": coords[3],
            "created_at": datetime.now().isoformat(),
            "urs_name": document_name,
            "section_name": current_section
        }
        results.append(result)
        block_number += 1

        x1, y1, x2, y2 = coords
        color = COLORS.get(block_type, (255, 255, 255))
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f"{block_type} #{block_number-1}", (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        del cropped

    annotated_path = os.path.join(RETRAINED_ANNOTATED_DIR, f"{Path(image_path).stem}_reannotated.png")
    cv2.imwrite(annotated_path, image, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    db_conn.commit()
    del image
    append_skipped_blocks_to_db(skipped_blocks)
    return results

10: Embedding and Database Storage

In [46]:
# Function to initialize embedding tools (SentenceTransformer and FAISS)
# In section "10: Embedding and Database Storage"
def initialize_embedding_tools(device='cpu'):
    global sentence_model, faiss_index
    if sentence_model is None:
        try:
            sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
            print("SentenceTransformer model loaded")
        except Exception as e:
            print(f"Error loading SentenceTransformer: {e}")
            sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
            print("Fallback to CPU for SentenceTransformer")

    if faiss_index is None:
        embedding_dim = sentence_model.encode("test").shape[0]
        faiss_index_path = FAISS_INDEX_PATH  # Use main index path
        if faiss_index_path.exists():
            try:
                faiss_index = faiss.read_index(str(faiss_index_path))
                print(f"Loaded existing FAISS index from {faiss_index_path} with {faiss_index.ntotal} embeddings")
            except Exception as e:
                print(f"Error loading FAISS index: {e}")
                faiss_index = None
        if faiss_index is None:
            faiss_index = faiss.IndexFlatL2(embedding_dim)
            print(f"FAISS index initialized with dimension {embedding_dim}")
    return faiss_index_path

# In section "10: Embedding and Database Storage"
def append_to_db(data):
    if not data:
        return

    setup_db_connection()
    global batch_run_id
    faiss_index_path = FAISS_INDEX_PATH  # Use main index path

    # Backup the existing FAISS index before modifying
    if faiss_index_path.exists():
        backup_path = FAISS_BACKUP_DIR / f"faiss_index_backup_{datetime.now().strftime('%Y%m%d')}.index"
        try:
            shutil.copy(faiss_index_path, backup_path)
            print(f"Backed up FAISS index to {backup_path}")
        except Exception as e:
            print(f"Error creating FAISS index backup: {e}")

    # Initialize or load the FAISS index
    if faiss_index is None:
        faiss_index_path = initialize_embedding_tools()

    try:
        separator = "|||"
        query_strings = [
            f"{d.get('urs_name', 'Unknown')}{separator}"
            f"{d.get('section_name', d['section_name'])}{separator}"
            f"{d['content_type']}{separator}"
            f"{d['content']}"
            for d in data if d.get('content_type') != 'title'
        ]
        embeddings = sentence_model.encode(query_strings, convert_to_numpy=True)

        faiss_index.add(embeddings)
        index_positions = list(range(faiss_index.ntotal - len(data), faiss_index.ntotal))

        values = [
            (d["source_file_run_id"], d["section_id"], d["block_number"], d["content_type"], d["content"],
             d["coord_x1"], d["coord_y1"], d["coord_x2"], d["coord_y2"], d["created_at"], idx)
            for d, idx in zip(data, index_positions)
        ]

        query = """
            INSERT INTO content_blocks (
                source_file_run_id, section_id, block_number, content_type, content,
                coord_x1, coord_y1, coord_x2, coord_y2, created_at, faiss_index_id
            ) VALUES %s
            ON CONFLICT (source_file_run_id, block_number) DO UPDATE SET
                section_id = EXCLUDED.section_id,
                content_type = EXCLUDED.content_type,
                content = EXCLUDED.content,
                coord_x1 = EXCLUDED.coord_x1,
                coord_y1 = EXCLUDED.coord_y1,
                coord_x2 = EXCLUDED.coord_x2,
                coord_y2 = EXCLUDED.coord_y2,
                created_at = EXCLUDED.created_at,
                faiss_index_id = EXCLUDED.faiss_index_id
        """
        execute_values(db_cursor, query, values)
        db_conn.commit()
        print(f"Inserted/Updated {len(data)} records into content_blocks for batch {batch_run_id}")

        faiss.write_index(faiss_index, str(faiss_index_path))
        print(f"Saved FAISS index to {faiss_index_path} with {faiss_index.ntotal} embeddings")
    except Exception as e:
        print(f"Database error: {e}")
        db_conn.rollback()
        raise

11a: Update COCO JSON Paths

In [47]:
# Function to update COCO JSON paths and save the result
def update_coco_paths(json_path, image_dir):
    updated_json_path = map_and_update_coco_paths(json_path, image_dir)
    print(f"COCO JSON paths updated and saved to {updated_json_path}")
    return updated_json_path

# Execute the COCO path update
try:
    updated_json_path = update_coco_paths(COCO_JSON_PATH, IMG_STAGING_DIR)
except Exception as e:
    print(f"Failed to update COCO JSON paths: {e}")
    print(traceback.format_exc())
    update_run_status('FAILED')
    raise

Actual images in /content/drive/My Drive/lifesciences/post_label/images: ['dba943c5-ProtonGlow_Test_URS_1_page_14.png', '32b16166-ProtonGlow_Test_URS_1_page_34.png', 'c77134ed-ProtonGlow_Test_URS_1_page_26.png', '2124bdf9-ProtonGlow_Test_URS_1_page_5.png', 'f7c3a644-ProtonGlow_Test_URS_1_page_16.png', 'f19c46f0-ProtonGlow_Test_URS_1_page_30.png', 'fe7aa29b-ProtonGlow_Test_URS_1_page_32.png', 'c8aa0be3-ProtonGlow_Test_URS_1_page_7.png', '042f030e-ProtonGlow_Test_URS_1_page_9.png', '3f57eebd-ProtonGlow_Test_URS_1_page_19.png', '7347d622-ProtonGlow_Test_URS_1_page_20.png', '245272e6-ProtonGlow_Test_URS_1_page_28.png', '426fb557-Pharma_URS_Enhanced_page_1.png', '5e5d7ccd-ProtonGlow_Test_URS_1_page_10.png', '68b58f6d-ProtonGlow_Test_URS_1_page_15.png', 'c9df1740-ProtonGlow_Test_URS_1_page_12.png', 'eddd6c38-ProtonGlow_Test_URS_1_page_29.png', '48759adc-Pharma_URS_Enhanced_page_2.png', '48ed3ccf-ProtonGlow_Test_URS_1_page_31.png', '148a3bf9-ProtonGlow_Test_URS_1_page_2.png', 'ea5776e7-Proton

11b: Train LayoutParser Model

In [48]:
# Function to train the model using updated COCO JSON
def train_model(updated_json_path, image_dir):
    train_layoutparser_model(updated_json_path, image_dir)
    print(f"Model training completed using {updated_json_path}")


11c: Load COCO Data

In [49]:
# Function to load the updated COCO JSON data
def load_coco_data(updated_json_path):
    with open(updated_json_path, "r") as f:
        coco_data = json.load(f)
    print(f"Loaded COCO data from {updated_json_path}")
    return coco_data

# Execute the COCO data loading
try:
    coco_data = load_coco_data(updated_json_path)
except Exception as e:
    print(f"Failed to load COCO data: {e}")
    #print(traceback.format_exc())
    update_run_status('FAILED')
    raise

Loaded COCO data from /content/drive/My Drive/lifesciences/retrained_staging/results_mapped.json


11d: Process Images and Annotate

In [50]:
# Function to process and annotate images
# Step 4: Process images and annotate

# In section "11d: Process Images and Annotate"
def process_images(coco_data):
    results = []
    source_files = [f for f in os.listdir(SOURCE_DIR) if f.endswith(('.docx', '.pdf'))]
    image_to_source_map = {}
    block_number_counter = {}  # Track block numbers per source_file_run_id

    # Build image-to-source mapping
    for source_file in source_files:
        source_base_name = Path(source_file).stem
        source_file_path = os.path.join(SOURCE_DIR, source_file)
        for image_info in coco_data["images"]:
            image_path = image_info["file_name"]
            image_base_name = Path(image_path).stem
            if source_base_name in image_base_name:
                image_to_source_map[image_path] = source_file_path

    # Process each image
    for image_info in coco_data["images"]:
        image_path = image_info["file_name"]
        if os.path.exists(image_path):
            original_file_path = image_to_source_map.get(image_path)
            if original_file_path and os.path.exists(original_file_path):
                print(f"Reannotating {image_path} with source file {original_file_path}")
                # Process layout and adjust block numbers
                layout_results = process_layout(image_path, original_file_path)
                if layout_results:
                    source_file_run_id = layout_results[0]["source_file_run_id"]
                    # Initialize counter for this source_file_run_id if not present
                    if source_file_run_id not in block_number_counter:
                        block_number_counter[source_file_run_id] = 0
                    # Adjust block numbers to be cumulative
                    for result in layout_results:
                        block_number_counter[source_file_run_id] += 1
                        result["block_number"] = block_number_counter[source_file_run_id]
                    results.extend(layout_results)
            else:
                print(f"No matching source file found for {image_path}, skipping")
        else:
            print(f"Image not found: {image_path}")
    print(f"process_images generated {len(results)} results")
    return results

# No change needed in process_layout, but ensure it uses block_number as assigned




11e: Append Results to Database

In [51]:
# Function to append results to the database
def save_results_to_db(results):
    append_to_db(results)
    print(f"Results appended to database")



11f: Update Run Status to Success

In [52]:
# Function to mark the run as successful
def mark_run_success():
    update_run_status('SUCCESS')
    print("Run status updated to SUCCESS")



12a: GPU Status Check

In [53]:
# Check GPU status using nvidia-smi
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print(result.stdout)
    else:
        print("nvidia-smi failed:", result.stderr)
except Exception as e:
    print(f"nvidia-smi not found or error: {e}")

nvidia-smi not found or error: [Errno 2] No such file or directory: 'nvidia-smi'


12b: Core Processing

In [ ]:
"""12b: Core Processing"""

"""12b: Core Processing"""

try:
    # Step 0: Ensure database connection is set up (sets status to RUNNING)
    setup_db_connection()

    # Step 1: Update COCO JSON paths
    updated_json_path = update_coco_paths(COCO_JSON_PATH, IMG_STAGING_DIR)

    # Step 2: Train the model
    # train_model(updated_json_path, IMG_STAGING_DIR)

    # Step 3: Load COCO data
    coco_data = load_coco_data(updated_json_path)

    # Step 4: Process images and annotate
    results = process_images(coco_data)

    print(results)
    # Step 5: Append results to database
    save_results_to_db(results)

    # Step 6: Mark run as successful
    mark_run_success()  # Sets status to SUCCESS
except Exception as e:
    print(f"Processing failed with unexpected error: {e}")
    print(traceback.format_exc())
    update_run_status('FAILED')  # Changed from 'FAILURE' to 'FAILED'
    raise

Connected to Neon PostgreSQL database
Started new batch run: run_number=3, batch_run_id=18, status=RUNNING
Actual images in /content/drive/My Drive/lifesciences/post_label/images: ['dba943c5-ProtonGlow_Test_URS_1_page_14.png', '32b16166-ProtonGlow_Test_URS_1_page_34.png', 'c77134ed-ProtonGlow_Test_URS_1_page_26.png', '2124bdf9-ProtonGlow_Test_URS_1_page_5.png', 'f7c3a644-ProtonGlow_Test_URS_1_page_16.png', 'f19c46f0-ProtonGlow_Test_URS_1_page_30.png', 'fe7aa29b-ProtonGlow_Test_URS_1_page_32.png', 'c8aa0be3-ProtonGlow_Test_URS_1_page_7.png', '042f030e-ProtonGlow_Test_URS_1_page_9.png', '3f57eebd-ProtonGlow_Test_URS_1_page_19.png', '7347d622-ProtonGlow_Test_URS_1_page_20.png', '245272e6-ProtonGlow_Test_URS_1_page_28.png', '426fb557-Pharma_URS_Enhanced_page_1.png', '5e5d7ccd-ProtonGlow_Test_URS_1_page_10.png', '68b58f6d-ProtonGlow_Test_URS_1_page_15.png', 'c9df1740-ProtonGlow_Test_URS_1_page_12.png', 'eddd6c38-ProtonGlow_Test_URS_1_page_29.png', '48759adc-Pharma_URS_Enhanced_page_2.png', 

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Skipping title inside list at coordinates: [354, 298, 2160, 572]
Skipping title inside table at coordinates: [391, 954, 2145, 1062]
Skipping title inside table at coordinates: [407, 1554, 1182, 1776]
Skipping title inside list at coordinates: [418, 2490, 935, 2705]
Inserted 4 skipped blocks into skipped_block_items
Reannotating /content/drive/My Drive/lifesciences/post_label/images/48759adc-Pharma_URS_Enhanced_page_2.png with source file /content/drive/My Drive/lifesciences/training_documents/Pharma_URS_Enhanced.docx
Processed source file metadata for /content/drive/My Drive/lifesciences/training_documents/Pharma_URS_Enhanced.docx: source_file_run_id = 311
Skipping title inside table at coordinates: [307, 280, 2108, 562]
Skipping title inside table at coordinates: [410, 1108, 1005, 1312]
Skipping title inside table at coordinates: [311, 1670, 2237, 1903]
Skipping text inside table at coordinates: [331, 2415, 2187, 2663]
Inserted 4 skipped blocks into skipped_block_items
Reannotating /c